In [229]:
import random
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [253]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dartmedia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [254]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [255]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [256]:
def stem(word):
    return stemmer.stem(word.lower())

In [257]:
def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)

    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [276]:
with open('dataset.json', 'r') as f:
    intents = json.loads(f.read())

In [277]:
all_words = []
tags = []
xy = []

In [278]:
for intent in intents.get('intents'):
    tag = intent.get('tag')
    tags.append(tag)
    for pattern in intent.get('patterns'):
        if pattern is None:
            continue
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [279]:
ignore_words = ['?', '!', '.', ',']

In [280]:
all_words = [stem(w) for w in all_words if w not in ignore_words]

In [281]:
all_words = sorted(set(all_words))

In [282]:
tags = sorted(set(tags))

In [283]:
X_train = []
y_train = []

In [284]:
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)

In [285]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [286]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [287]:
batch_size = 8

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          num_workers=0)

In [288]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [289]:
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [290]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [291]:
learning_rate = 0.001
num_epochs = 1000

In [292]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [293]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        
        output = model(words)
        loss = criterion(output , labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}')

print(f'final loss, loss={loss.item():.4f}')

epoch 100/1000, loss=0.4807
epoch 200/1000, loss=0.0316
epoch 300/1000, loss=0.0020
epoch 400/1000, loss=0.0037
epoch 500/1000, loss=0.0011
epoch 600/1000, loss=0.2355
epoch 700/1000, loss=0.0009
epoch 800/1000, loss=0.0003
epoch 900/1000, loss=0.4625
epoch 1000/1000, loss=0.0003
final loss, loss=0.0003


In [294]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

In [297]:
FILE = "data.pth"
torch.save(data, FILE)

In [298]:
data = torch.load(FILE)

In [299]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

In [300]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=117, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=18, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.1:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: halo
Sam: Halo. Saya ingin menginformasikan bahwa tagihan kartu kredit {bank_kredit} atas nama {panggilan} {nama} telah jatuh tempo di tanggal {bulan_tagihan} sebesar {total_tagihan}. Apakah bisa segera dilunasi {panggilan}? 
You: saya udah bayar hari senin jam 10 di ATM
Sam: Dalam sistem kami, {panggilan} masih mempunyai tagihan yang belum lunas di {bulan_tagihan} dengan jumlah {total_tagihan}. Boleh mohon diinfokan hari, jam, dan metode pembayarannya?
